In [22]:
#| default_exp core

# pytrim2

> A python program for trimming and demultiplexing nanopore reads.

In [34]:
#| hide
from nbdev.showdoc import *

In [35]:
#| export

# Import dependencies
from Bio import SeqIO
from Bio import Align
from Bio.Seq import Seq
import numpy as np

In [36]:
#| export

def findAlingments(seq_record, primer_dict, inward_end, max_alignments):
    "Find alignments for each primer in a sequence record"
    primer_keys = list(primer_dict.keys())
  
    aligner = Align.PairwiseAligner()
    aligner.match_score = 1.0
    aligner.mismatch_score = 0
    aligner.gap_score = -2
    aligner.mode = "local"

    n_sequences = len(primer_keys)

    array_cols = max_alignments + 3
    al_array = np.zeros( (n_sequences, array_cols) )

    for i in list(range(0, n_sequences, 1)):
        al = []
        seq = primer_dict[primer_keys[i]].seq        
        alignments = aligner.align(seq_record[0:inward_end], seq)
        len_alignments = len(alignments)
        if(len_alignments <= max_alignments):
            score = alignments.score
            al = [j.aligned for j in alignments]
            len_al = len(al)
            for k in range(0, len_al):
                al[k] = (al[k][0][0][1])
            al_array[i, 0:len(al)] = al # ends of each alignment
            al_array[i, -3] = max(al) # maximum posistion of each alignment
            al_array[i, -2] = len_alignments # number of alingments
            al_array[i, -1] = np.around(alignments.score/len(seq)*100, 0) # normalized local alingnment score
            
    return(al_array)
    
    

In [37]:
#| hide

primer_dict = SeqIO.index("test_data/test_primer.fasta", "fasta")
seq_record = Seq("TGATGTAAGTACGCTCAGTTCGATATCGATATGAGACGGATTAGGAGGGGGCGCGATGTTGTGTGGGAAAA")
ends = findAlingments(seq_record, primer_dict, 200, 3)

print(ends)

[[ 21.   0.   0.  21.   1. 100.]
 [ 37.   0.   0.  37.   1.  60.]
 [ 38.   0.   0.  38.   1. 100.]
 [  8.   0.   0.   8.   1.  60.]
 [  0.   0.   0.   0.   0.   0.]
 [ 60.   0.   0.  60.   1. 100.]]


In [38]:
#| export

def align_barcodes(primer_dict, record_dict, inward_end, max_alignments):
    "Aligne all barcodes in a list of seq records"
    
    record_keys = list(record_dict.keys())
    n_sequences = len(record_keys)
    
    alingments = list( range(0, n_sequences) )
    for i in list(range(0, n_sequences, 1)):
        seq_i = record_dict[record_keys[i]].seq
    
        alingments[i] = findAlingments(seq_i, primer_dict, inward_end, max_alignments)
    return(alingments)

In [39]:
#| hide

primer_dict = SeqIO.index("test_data/test_primer.fasta", "fasta")
record_dict = SeqIO.index("test_data/test.fasta", "fasta")
alginment_arrays = align_barcodes(primer_dict, record_dict, 200, 5)
alginment_arrays

[array([[ 24.,   0.,   0.,   0.,   0.,  24.,   1., 100.],
        [ 43., 197.,   0.,   0.,   0., 197.,   2.,  60.],
        [ 75., 100., 103., 109.,   0., 109.,   4.,  50.],
        [ 94., 189.,   0.,   0.,   0., 189.,   2.,  50.],
        [ 27., 103., 118.,   0.,   0., 118.,   3.,  60.],
        [ 18.,   0.,   0.,   0.,   0.,  18.,   1.,  70.]]),
 array([[ 16.,  59.,  72., 153.,   0., 153.,   4.,  60.],
        [ 31.,   0.,   0.,   0.,   0.,  31.,   1., 100.],
        [  8.,  29.,   0.,   0.,   0.,  29.,   2.,  60.],
        [ 76.,   0.,   0.,   0.,   0.,  76.,   1.,  70.],
        [ 30.,  86., 198.,   0.,   0., 198.,   3.,  60.],
        [ 14.,  22.,  56.,  89., 177., 177.,   5.,  50.]]),
 array([[149., 156., 179.,   0.,   0., 179.,   3.,  60.],
        [ 28.,  97., 186.,   0.,   0., 186.,   3.,  60.],
        [ 29.,   0.,   0.,   0.,   0.,  29.,   1., 100.],
        [126.,   0.,   0.,   0.,   0., 126.,   1.,  70.],
        [108., 117.,   0.,   0.,   0., 117.,   2.,  60.],
        [ 

In [40]:
#| hide

x = np.zeros( (1, len(alginment_arrays)) )
for i in alginment_arrays:
    print(np.sum((i[:,-1] >= 85) & (i[:,-2] == 1)))
    print(i[np.where(i[:,-1] >= 85),:])

1
[[[ 24.   0.   0.   0.   0.  24.   1. 100.]]]
1
[[[ 31.   0.   0.   0.   0.  31.   1. 100.]]]
1
[[[ 29.   0.   0.   0.   0.  29.   1. 100.]]]


In [41]:
#| slow

primer_dict = SeqIO.index("test_data/test_primer.fasta", "fasta")
record_dict = SeqIO.index("test_data/test.fastq", "fastq")
alginment_arrays = align_barcodes(primer_dict, record_dict, 200, 10)

for i in alginment_arrays:
    print(np.sum(i[:,-1] >= 85))
    print(i[np.where(i[:,-1] >= 85),:])

FileNotFoundError: [Errno 2] No such file or directory: 'test_data/test.fastq'

In [71]:
#| export

def decide_barcode_id(alginment_arrays):
    "Decide which barcode is best hit; remove if tie"
    
    id_array = np.zeros((np.shape(alginment_arrays)[0],3), dtype=np.int64)
    for i in range(0, np.shape(alginment_arrays)[0]):
        array_i = alginment_arrays[i]
        id_i = np.where(array_i[:,-1] == np.max(array_i[:,-1]))[0]
        if len(id_i) == 1:
            id_array[i,0] = id_i
            id_array[i,1] = array_i[id_i,-3]
            id_array[i,2] = array_i[id_i,-1]
        elif len(id_i) >= 1:
            id_array[i,0] =  -1
            id_array[i,1] = 0
            id_array[i,2] = 0
        
        
    return(id_array)

In [63]:
#| hide

seq_barcode_ids = decide_barcode_id(alginment_arrays)
seq_barcode_ids

array([[  0,  24, 100],
       [  1,  31, 100],
       [  2,  29, 100]])

In [44]:
#| export

# Trim barcodes of sequence
def trim_record(seq_record, primer_end_position):
    x = seq_record
    x =  x[primer_end_position:]
    return(x)

In [64]:
#| hide

record_keys = list(record_dict.keys())
record_x = record_dict[record_keys[0]]
len_old = len(record_x.seq)

record_x_new = trim_record(record_x, 3)

(len(record_x_new.seq) + 3) == len_old


True

In [73]:
#| export

def sort_records_to_file(record_dict, primer_dict, output_folder, alginment_arrays, input_file_type):
    "Sort records into new files based on barcodes and name files after barcodes"
    
    seq_barcode_res = decide_barcode_id(alginment_arrays)
    seq_barcode_ids = seq_barcode_res[:,0]
    seq_barcode_end_pos = seq_barcode_res[:,1]
    seq_barcode_match = seq_barcode_res[:,2]
    primer_keys = list(primer_dict.keys())
    record_keys = list(record_dict.keys())
    record_numbers = range(0, len(record_keys))

    for k in range(0, len(primer_dict)):
        seq_iterator_k = (trim_record(record_dict[record_keys[i]], seq_barcode_end_pos[i]) for i in record_numbers if seq_barcode_ids[i] == k if seq_barcode_match[i] >= 85)
        SeqIO.write(seq_iterator_k, output_folder + "/" + primer_dict[primer_keys[k]].name + "_seqs." + input_file_type, input_file_type)

    

In [75]:
#| hide

sort_records_to_file(record_dict, primer_dict, "test_data/test_out", alginment_arrays, "fasta")

In [76]:
#| export

def demultiplex(input_file, input_file_type, primer_file, primer_file_type, output_folder, max_distance, max_alignments):
    "Run the program"
    
    primer_dict = SeqIO.index(primer_file, primer_file_type)
    record_dict = SeqIO.index(input_file, input_file_type)
    alginment_arrays = align_barcodes(primer_dict, record_dict, max_distance, max_alignments)
    sort_records_to_file(record_dict, primer_dict, output_folder, alginment_arrays, input_file_type)

In [77]:
#| hide
demultiplex("test_data/test.fasta", "fasta", "test_data/test_primer.fasta", "fasta", "test_data/test_out", 200, 5)

In [78]:
#| hide
demultiplex("/dss/dsshome1/lxc0F/ru75jul2/testcat/out/all.fastq", "fastq", "test_data/test_primer.fasta", "fasta", "/dss/dsshome1/lxc0F/ru75jul2/testcat/out/testout", 200, 5)

In [ ]:
#| slow

# mock
demultiplex("/dss/dsshome1/lxc0F/ru75jul2/seq_data/all_mock.fastq", "fastq", "test_data/test_primer.fasta", "fasta", "/dss/dsshome1/lxc0F/ru75jul2/seq_data/adv_out/mock/", 200, 3)

In [ ]:
#| slow

# wt
demultiplex("/dss/dsshome1/lxc0F/ru75jul2/seq_data/all_wt.fastq", "fastq", "test_data/test_primer.fasta", "fasta", "/dss/dsshome1/lxc0F/ru75jul2/seq_data/adv_out/wt/", 200, 3)